In [ ]:
import requests
import time
import pandas as pd
import datetime as dt

In [ ]:
def query_pushshift(subreddit, kind = 'submission', day_window = 7, n = 520):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500" # always pulling max of 500
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(1.5)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full.reset_index(drop=True)

In [ ]:
# Saving the returned results to a variable
results = query_pushshift('marvel')

In [ ]:
# Checking how many posts were scraped
results.shape

In [ ]:
results1 = query_pushshift('marvelstudios')

In [ ]:
results1.shape

In [ ]:
# Saving my results to a csv
results.to_csv('marvel_reddit.csv'),results1.to_csv('marvelstudios_reddit.csv')